In [ ]:
names = ["mimic-biovil-frontal", "chexpertplus-gloria-frontal"]
sections = ["findings", "impression"]
mlms = ["0.0", "0.15"]
loss_combos = ["ig_tg", "ig_tgl", "igl_tg", "igl_tgl"]

In [ ]:
template = """LOSS_COMBO={loss_combo}
SECTION={section}
NAME={name}
MLM_P={mlm_prob}

is_zero=$(echo "$MLM_P == 0.0" | bc)

if [[ "$is_zero" -eq 1 ]]; then
    MLM="no-mlm"
else
    MLM="mlm"
fi

VERSION=$SECTION-$LOSS_COMBO-$MLM

python train.py fit \\
--config configs/$NAME.yaml \\
--model.checkpoint_path /opt/gpudata/remix/$NAME-$VERSION \\
--model.loss_combo $LOSS_COMBO \\
--data.section $SECTION \\
--trainer.logger.name $NAME \\
--trainer.logger.version $VERSION
"""

In [ ]:
exps = [{
    "loss_combo": lc,
    "section": s,
    "name": n,
    "mlm_prob": m,
} for n in names for s in sections for m in mlms for lc in loss_combos]

In [ ]:
# write individual experiment scripts
for exp in exps:
    script = template.format(**exp)
    n = exp["name"]
    s = exp["section"]
    lc = exp["loss_combo"].replace("_", "-")
    m = "mlm" if float(exp["mlm_prob"]) != 0 else "no-mlm"
    fname = f"train-{n}-{s}-{lc}-{m}"
    with open(f"scripts/{fname}.sh", "w") as f:
        f.write(script)

In [ ]:
# write batched jobs
for n, s, m in [(n, s, m) for n in names for s in sections for m in mlms]:
    m = 'mlm' if float(m) != 0 else 'no-mlm'
    with open(f"run-{n}-{s}-{m}.sh", "w") as f:
        for lc in loss_combos:
            line = f"bash scripts/train-{n}-{s}-{lc.replace('_', '-')}-{m}.sh\n"
            f.write(line)